# Trento数据集
MFT论文中采用的是disjoint划分，但这个数据集我只找到了整张图片的数据

此代码中的划分采用了sample_gt函数 每类选择184个测试样本的划分方式。


In [1]:
import scipy.io as io
import numpy as np


In [2]:
data = io.loadmat(r'trento_data.mat')
print(data)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sat Mar 26 15:20:14 2022', '__version__': '1.0', '__globals__': [], 'HSI_data': array([[[1137344512, 1139245056, 1140031488, ..., 1151533056,
         1153990656, 1153679360],
        [1140867072, 1139965952, 1137704960, ..., 1151942656,
         1152204800, 1152671744],
        [1138884608, 1137311744, 1136689152, ..., 1149673472,
         1152204800, 1154195456],
        ...,
        [1136263168, 1139965952, 1139834880, ..., 1155063808,
         1156177920, 1158963200],
        [1136721920, 1138196480, 1138065408, ..., 1152737280,
         1153802240, 1155104768],
        [1141932032, 1135607808, 1136787456, ..., 1153646592,
         1153556480, 1156120576]],

       [[1138262016, 1139736576, 1134592000, ..., 1152524288,
         1155203072, 1153916928],
        [1135280128, 1134067712, 1137573888, ..., 1149739008,
         1150066688, 1153966080],
        [1135706112, 1136197632, 1136394240, ..., 1150107648,
      

In [3]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'HSI_data', 'LiDAR_data', 'ground'])

In [4]:
hsi = data['HSI_data']
hsi.shape

(166, 600, 63)

In [5]:
lidar = data['LiDAR_data']

lidar = np.expand_dims(lidar, axis=2)
lidar.shape

(166, 600, 1)

In [6]:
ground = data['ground']
ground.shape


(166, 600)

In [7]:
import sklearn
import sklearn.model_selection
def sample_gt(gt, train_size, mode='random'):
    if train_size==1:
        return gt
    """Extract a fixed percentage of samples from an array of labels.

    Args:
        gt: a 2D array of int labels
        percentage: [0, 1] float
    Returns:
        train_gt, test_gt: 2D arrays of int labels

    """
    indices = np.nonzero(gt)
    X = list(zip(*indices)) # x,y features
    y = gt[indices].ravel() # classes
    train_gt = np.zeros_like(gt)
    test_gt = np.zeros_like(gt)
    if train_size > 1:
       train_size = int(train_size)

    if mode == 'random':
       train_indices, test_indices = sklearn.model_selection.train_test_split(X, train_size=train_size, stratify=y)
       train_indices = tuple(list(t) for t in zip(*train_indices))
       test_indices = tuple(list(t) for t in zip(*test_indices))
       train_gt[train_indices] = gt[train_indices]
       test_gt[test_indices] = gt[test_indices]
    elif mode == 'fixed':
       print("Sampling {} with train size = {}".format(mode, train_size))
       train_indices, test_indices = [], []
       for c in np.unique(gt):
           if c == 0:
              continue
           indices = np.nonzero(gt == c)
           X = list(zip(*indices)) # x,y features

           train, test = sklearn.model_selection.train_test_split(X, train_size=train_size)
           train_indices += train
           test_indices += test
       train_indices = [list(t) for t in zip(*train_indices)]
       test_indices = [list(t) for t in zip(*test_indices)]
       train_gt[train_indices] = gt[train_indices]
       test_gt[test_indices] = gt[test_indices]

    elif mode == 'disjoint':
        train_gt = np.copy(gt)
        test_gt = np.copy(gt)
        for c in np.unique(gt):
            mask = gt == c
            for x in range(gt.shape[0]):
                first_half_count = np.count_nonzero(mask[:x, :])
                second_half_count = np.count_nonzero(mask[x:, :])
                try:
                    ratio = first_half_count / (first_half_count + second_half_count)
                    if ratio > 0.9 * train_size:
                        break
                except ZeroDivisionError:
                    continue
            mask[:x, :] = 0
            train_gt[mask] = 0

        test_gt[train_gt > 0] = 0
    else:
        raise ValueError("{} sampling is not implemented yet.".format(mode))
    return train_gt,test_gt
def choose_true_point(true_data, num_classes):
    number_true = []
    pos_true = {}
    for i in range(num_classes + 1):
        each_class = np.argwhere(true_data == i)
        number_true.append(each_class.shape[0])
        pos_true[i] = each_class

    total_pos_true = pos_true[0]
    for i in range(1, num_classes + 1):
        total_pos_true = np.r_[total_pos_true, pos_true[i]]
    total_pos_true = total_pos_true.astype(int)
    return total_pos_true, number_true


def choose_img_point(height, width):
    total_pos_true = np.array([[i, j] for i in range(height) for j in range(width)])
    return total_pos_true


# 1
def chooose_point(test_data, num_classes):
    number_test = []
    pos_test = {}

    for i in range(num_classes):
        each_class = np.argwhere(test_data == (i + 1))
        number_test.append(each_class.shape[0])
        pos_test[i] = each_class

    total_pos_test = pos_test[0]
    for i in range(1, num_classes):
        total_pos_test = np.r_[total_pos_test, pos_test[i]]  # (9671,2)
    total_pos_test = total_pos_test.astype(int)
    return total_pos_test, number_test


def mirror_hsi( input_normalize, patch=5):
    height, width, band = input_normalize.shape
    padding = patch // 2
    mirror_hsi = np.zeros((height + 2 * padding, width + 2 * padding, band), dtype=float)  # padding后的图 上下左右各加padding

    mirror_hsi[padding:(padding + height), padding:(padding + width), :] = input_normalize  # 中间用原图初始化

    for i in range(padding):
        mirror_hsi[padding:(height + padding), i, :] = input_normalize[:, padding - i - 1, :]

    for i in range(padding):
        mirror_hsi[padding:(height + padding), width + padding + i, :] = input_normalize[:, width - 1 - i, :]

    for i in range(padding):
        mirror_hsi[i, :, :] = mirror_hsi[padding * 2 - i - 1, :, :]

    for i in range(padding):
        mirror_hsi[height + padding + i, :, :] = mirror_hsi[height + padding - 1 - i, :, :]

    print("**************************************************")
    print("patch is : {}".format(patch))
    print("mirror_image shape : [{0},{1},{2}]".format(mirror_hsi.shape[0], mirror_hsi.shape[1], mirror_hsi.shape[2]))
    print("**************************************************")
    return mirror_hsi


def gain_neighborhood_pixel(mirror_image, point, i, patch=5):
    x = point[i, 0]
    y = point[i, 1]
    temp_image = mirror_image[x:(x + patch), y:(y + patch), :]
    return temp_image




def get_data(mirror_image, test_point, patch=5):
    height, width, band = mirror_image.shape
    x_test = np.zeros((test_point.shape[0], patch, patch, band), dtype=float)

    for j in range(test_point.shape[0]):
        x_test[j, :, :, :] = gain_neighborhood_pixel(mirror_image, test_point, j, patch)
    print("x_test  shape = {}, type = {}".format(x_test.shape, x_test.dtype))
    print("**************************************************")

    return x_test


def get_label(number_test, num_classes):
    y_test = []
    for i in range(num_classes):
        for k in range(number_test[i]):
            y_test.append(i+1)

    y_test = np.array(y_test)
    print("y_test: shape = {} ,type = {}".format(y_test.shape, y_test.dtype))
    print("**************************************************")
    return y_test

def norm(data):
    input_normalize = np.zeros(data.shape)

    for i in range(data.shape[2]):
        input_max = np.max(data[:, :, i])
        input_min = np.min(data[:, :, i])
        input_normalize[:, :, i] = (data[:, :, i] - input_min) / (input_max - input_min)
    return input_normalize

In [8]:
train_gt,test_gt = sample_gt(ground,184)

In [9]:
np.nonzero(ground)[0]

array([  0,   0,   0, ..., 164, 165, 165])

In [10]:
num_classes = 6

#
total_pos_test, number_test = chooose_point(test_gt, num_classes)
total_pos_train, number_train = chooose_point(train_gt, num_classes)

In [11]:
len(total_pos_train)

184

In [12]:
hsi = norm(hsi)
io.savemat(r'HSI_norm.mat',{'Data':hsi})
hsi = mirror_hsi(hsi,11)

HSI_Te = get_data(hsi, total_pos_test,patch=11,)
io.savemat("HSI_Te.mat",{"Data":HSI_Te})
del HSI_Te


HSI_Tr = get_data(hsi, total_pos_train,patch=11,)
io.savemat("HSI_Tr.mat",{"Data":HSI_Tr})
del HSI_Tr
del  hsi

**************************************************
patch is : 11
mirror_image shape : [176,610,63]
**************************************************
x_test  shape = (30030, 11, 11, 63), type = float64
**************************************************
x_test  shape = (184, 11, 11, 63), type = float64
**************************************************


In [13]:
lidar = norm(lidar)
io.savemat(r'LIDAR_norm.mat',{'Data':lidar})
lidar = mirror_hsi(lidar,11)

LIDAR_Te = get_data(lidar, total_pos_test,patch=11,)
io.savemat("LIDAR_Te.mat",{"Data":LIDAR_Te})
del LIDAR_Te


LIDAR_Tr = get_data(lidar, total_pos_train,patch=11,)
io.savemat("LIDAR_Tr.mat",{"Data":LIDAR_Tr})

**************************************************
patch is : 11
mirror_image shape : [176,610,1]
**************************************************
x_test  shape = (30030, 11, 11, 1), type = float64
**************************************************
x_test  shape = (184, 11, 11, 1), type = float64
**************************************************


In [14]:
TeLabel = get_label(number_test, num_classes)
TrLabel = get_label(number_train, num_classes)
io.savemat("TeLabel.mat",{"Data":TeLabel})
io.savemat("TrLabel.mat",{"Data":TrLabel})

y_test: shape = (30030,) ,type = int64
**************************************************
y_test: shape = (184,) ,type = int64
**************************************************
